In [5]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import struct

from render import initialize_scene
from gaussian_splatting import GaussianScene
from utils import in_view_frustum

In [6]:
def export_culled_gaussians(scene: GaussianScene, filename: str, image_num: int):

    image = scene.images[image_num]  

    # Compute visibility mask
    in_view = in_view_frustum(
        points=scene.gaussians.points,
        view_matrix=image.world_view_transform
    )

    # Extract only culled Gaussians (inside frustum)
    points = scene.gaussians.points[in_view]
    covariance_3d = scene.gaussians.get_3d_covariance_matrix()[in_view]
    colors = scene.gaussians.colors[in_view]
    opacities = torch.sigmoid(scene.gaussians.opacity[in_view])  

    # Convert to camera space and extract depth values 
    points_homogeneous = torch.cat([points, torch.ones(points.shape[0], 1)], dim=1)
    points_camera = (points_homogeneous @ image.world_view_transform)[:, :3]  
    depths = points_camera[:, 2]  

    # Sort Gaussians by depth (back to front for alpha blending)
    sorted_indices = torch.argsort(depths, descending=True)

    points = points[sorted_indices].cpu().detach().numpy()
    covariance_3d = covariance_3d[sorted_indices].cpu().detach().numpy()
    colors = colors[sorted_indices].cpu().detach().numpy()
    opacities = opacities[sorted_indices].cpu().detach().numpy()

    with open(filename, 'wb') as f:
        for i in range(len(points)):
            # Gaussian position (3D)
            x, y, z = points[i]

            # Gaussian color (RGB)
            r, g, b = colors[i]

            # Gaussian covariance (3x3 matrix flattened)
            covariance = covariance_3d[i].flatten()

            # Gaussian opacity
            opacity = opacities[i]

            # Write to binary file
            f.write(struct.pack('fff', x, y, z))  
            f.write(struct.pack('fff', r, g, b))   
            f.write(struct.pack('fffffffff', *covariance))  
            f.write(struct.pack('f', opacity))   

    print(f"Saved {len(points)} culled Gaussians to {filename}")


In [12]:
def export_camera(scene: GaussianScene, filename: str, image_num: int):

    image = scene.images[image_num]  

    view_matrix = image.world_view_transform.cpu().numpy()
    projection_matrix = image.projection_matrix.cpu().numpy()

    width, height = int(image.width.item()), int(image.height.item())

    with open(filename, 'wb') as f:
        f.write(view_matrix.tobytes())   
        f.write(projection_matrix.tobytes())   
        f.write(struct.pack('ii', width, height))  

    print(f"Saved camera data to {filename}")


In [13]:
colmap_path = "../data/360-extra/treehill/sparse/0"
image_num = 100  # Index of the image to render

# Initialize scene
scene = initialize_scene(colmap_path)

In [14]:
export_culled_gaussians(scene=scene, filename='./assets/sorted_culled_gaussians.bin', image_num=image_num)

/tmp/ipykernel_232808/2431262533.py:48: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  f.write(struct.pack('f', opacity))


Saved 46050 culled Gaussians to ./assets/sorted_culled_gaussians.bin


In [15]:
export_camera(scene, "./assets/camera.bin", image_num=image_num)


Saved camera data to ./assets/camera.bin
